<a href="https://colab.research.google.com/github/vin-thomas/CNN_Text_Classification/blob/master/Enron_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import torch
import sklearn
from IPython.core.display import display, HTML
import glob
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 596 kB 37.9 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 6.8 MB 41.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
from transformers import BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/AIML Labs/@Project/working files 2b/Pre_processed/enron_csv2', index_col=[0])

In [ ]:
df

,Name,Folder,From,To,CC,BCC,date,Subject,Mail_id,Body
0,sager-e,notes_inbox,enron.announcements@enron.com,all.states@enron.com,1000,1000,2000-12-12 00:00:00,improved process for engaging temporary worker...,3.206950e+07,"esager.nsfrnrnas you are aware, enron utilize..."
1,sager-e,notes_inbox,veronica.gonzalez@enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,margin letter socalrnmimeversion: .rncontentt...,3.595780e+06,esager.nsfrnrnattached is the margin letter o...
2,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,your approval is overdue: access request for s...,1.737167e+07,esager.nsfrnrnthis request has been pending y...
3,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,2001-06-07 00:00:00,your approval is overdue: access request forrn...,3.157035e+07,esager.nsfrnrnthis request has been pending y...
4,sager-e,notes_inbox,lech.kalembka@cwt.com,elizabeth.sager@enron.com,david.mitchell@cwt.com,david.mitchell@cwt.com,2001-05-04 00:00:00,setoff provisionsrncc: david.mitchell@cwt.comr...,8.588396e+06,esager.nsfrnrnwe have attached comments on th...
...,...,...,...,...,...,...,...,...,...,...
516107,hain-m,all_documents,christopher.behney@enron.com,toni.schulenburg@enron.com,mary.hain@enron.com,mary.hain@enron.com,1998-05-27 00:00:00,re: lexis requestrncc: mary.hain@enron.comrnmi...,2.537982e+07,maryhain.nsfrnrnmary hainn sw salmon stn worl...
516275,hain-m,all_documents,mary.hain@enron.com,alan.comnes@enron.com,1000,1000,2001-01-08 00:00:00,re: governor davis letterrnmimeversion: .rncon...,1.434573e+07,maryhain.nsfrnrnpithy.nnnnnalan comnesn// : a...
516325,hain-m,all_documents,mary.hain@enron.com,alan.comnes@enron.com,lysa.akin@enron.com,lysa.akin@enron.com,2000-12-28 00:00:00,re: my new deskrncc: lysa.akin@enron.comrnmime...,1.452668e+07,maryhain.nsfrnrnyes please. thanks.nnnnnalan...
516580,hain-m,all_documents,paul.kaufman@enron.com,mary.hain@enron.com,1000,1000,1999-06-25 00:00:00,re: federal restructuring legislationrnmimever...,1.381135e+07,"maryhain.nsfrnrna few thoughts:nnfirst, i thi..."


In [ ]:
df['Text'] = df['Subject']+ df['Body']

In [ ]:
df.nunique()

Name          226
Folder       1331
From        20282
To          19429
CC           8198
BCC          8051
date         1444
Subject    289784
Mail_id    289787
Body       289364
Text       289784
dtype: int64

In [ ]:
labels = df.Folder
content = df.Text

In [ ]:
mod_content =[]
for item in content:
  item= str(item)
  mod_content.append(item)

In [ ]:
type(mod_content), type(mod_content[0]), len(mod_content), type(labels), len(labels)

(list, str, 289792, pandas.core.series.Series, 289792)

In [ ]:
labels, uniques = pd.factorize(labels)

In [ ]:
type(labels)

numpy.ndarray

In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(mod_content, labels, test_size=0.1, random_state=42)

In [ ]:
train_df = pd.DataFrame(list(zip(train_inputs, train_labels)), columns =['Text', 'Labels'])
val_df = pd.DataFrame(list(zip(val_inputs,val_labels)), columns= ['Text', 'Labels'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
max_len = 150

In [ ]:
class BBC(Dataset):

  def __init__(self, df, tokenizer, max_len):
    self.text = df.Text
    self.targets = df.Labels
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, item):
    text = str(self.text[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors= 'pt'
      
    )

    return {
      'review_text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = BBC(
    df= df,
    tokenizer=tokenizer,
    max_len=max_len
    )

  return DataLoader(
      ds,
      batch_size=batch_size,
      shuffle = True,
      num_workers=2
      )

In [ ]:
train_df.shape, val_df.shape

((260812, 2), (28980, 2))

In [ ]:
val_df.Labels.unique()

In [ ]:
BATCH_SIZE = 50
val_batch_size =20
train_data_loader = create_data_loader(train_df, tokenizer, max_len, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, max_len, val_batch_size)

In [ ]:
data = next(iter(val_data_loader))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
from torch import nn
from transformers import BertModel, BertForSequenceClassification

class BertClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertClassifier, self).__init__()
    self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                              num_labels=1331,
                                                              output_attentions= False,
                                                              output_hidden_states= False)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      labels= labels
    )
           
    return pooled_output

In [ ]:
model = BertClassifier(5)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=1e-5)
device

device(type='cpu')

In [ ]:
model = model.to(device)

In [ ]:
def evaluate ():
  eval_model = model
  eval_model.eval
  val_accuracy = []
  val_loss = []
  
  for batch in val_data_loader:
    
    input_ids = batch['input_ids'].to(device, dtype= torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
    labels_ = batch['targets']
    labels = torch.nn.functional.one_hot(labels_, 5).squeeze(1)
    labels = labels.to(device, dtype= torch.float)
    
    result = model(input_ids= input_ids, attention_mask=attention_mask, 
                   labels=labels)
    loss = result.loss
    val_loss.append(loss.item())
    
    logits = result.logits

    _, predicted = torch.max(logits, 1)
         
    accuracy = (predicted == labels_.to(device)).cpu().numpy().mean() * 100
    #correct += (predicted == labels_.to(device)).sum().item()
    val_accuracy.append(accuracy)

  val_loss = np.mean(val_loss)
  val_accuracy = np.mean(val_accuracy)

  return val_loss, val_accuracy

In [ ]:
epochs=3
train_losses = []
train_accuracy = []
model.train()


for e in range(epochs):
  train_loss = 0
  correct = 0
  i=0
  for batch in (train_data_loader):
    i+=1
    input_ids = batch['input_ids'].to(device, dtype= torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
    labels_ = batch['targets']
    labels = torch.nn.functional.one_hot(labels_, 1331).squeeze(1)
    labels = labels.to(device, dtype= torch.float) 
    
    model.zero_grad()
 
    result = model(input_ids= input_ids, attention_mask=attention_mask, 
                   labels=labels)
    loss = result.loss
    logits = result.logits
      
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(),1)
    optimizer.step()

    _, predicted = torch.max(logits, 1)
    correct += (predicted == labels_.to(device)).sum().item()

    print(i,')', 'loss', loss.item(), 'correct', (predicted == labels_.to(device)).sum().item())
    
  
  train_losses.append(loss/len (train_inputs))
  train_accuracy.append(100 * correct/len(train_inputs))
  print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(e+1,train_losses[-1], train_accuracy[-1]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
model.eval()
val_accuracy = []
val_loss = []
  
for batch in val_data_loader:

  input_ids = batch['input_ids'].to(device, dtype= torch.long)
  attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
  labels_ = batch['targets']
  labels = torch.nn.functional.one_hot(labels_, 1331).squeeze(1)
  labels = labels.to(device, dtype= torch.float)
  
  result = model(input_ids= input_ids, attention_mask=attention_mask, 
                  labels=labels)
  loss = result.loss
  
  val_loss.append(loss.item())
  
  logits = result.logits

  _, predicted = torch.max(logits, 1)
        
  accuracy = (predicted == labels_.to(device)).cpu().numpy().mean() * 100
  val_accuracy.append(accuracy)


val_loss = np.mean(val_loss)
val_accuracy = np.mean(val_accuracy)
print ('validation_loss:', val_loss, 'validation accuracy:', val_accuracy)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

validation_loss: 0.1634738283852736 validation accuracy: 95.0


In [ ]:
torch.cuda.empty_cache()